In [422]:
import numpy as np
import matplotlib.pyplot as plt

def empuxo(RPM, d,pitch, V0, numero_de_motores):
# O empuxo de um propeller dado um motor que fornece uma rotacação RPM
# V0 = propeller forward airspeed , velocidade da hélice para a frente 

    return ((4.392399/10**(8))*RPM*((d**(3.5))/np.sqrt(pitch))*(4.23333*10**(-4)*RPM*pitch - V0))*numero_de_motores

def empuxo_dada_a_tensao(V,KV, d,pitch, V0, numero_de_motores):
    RPM = V*KV
    return ((4.392399/10**(8))*RPM*((d**(3.5))/np.sqrt(pitch))*(4.23333*10**(-4)*RPM*pitch - V0))*numero_de_motores

def V0_dado_um_teste_de_fabricante(F,d, pitch, v, KV):
    RPM = KV*v
    #-V0 = (F/((4.392399*10**-8)*RPM*(d**3.5)/np.sqrt(pitch))) - 4.2*(10**-4)*RPM*pitch
    V0 = -(F/((4.392399*10**-8)*RPM*(d**3.5)/np.sqrt(pitch))) + 4.2*(10**-4)*RPM*pitch
    return V0

# ============== Dimensionamento =========================================
# 1) diâmetro_max_propeller = aplicação(massa, tamanho do frame, taxa Empuxo-peso)
# 2)
def RPM(P,trust_to_weight_ratio , d,pitch, V0, numero_de_motores):
    F = P*trust_to_weight_ratio
    a = (4.392399/10**(8))*((d**(3.5))/np.sqrt(pitch))*(4.23333*10**(-4))*pitch
    b = -(4.392399/10**(8))*((d**(3.5))/np.sqrt(pitch))*V0
    c = - F/numero_de_motores
    # 0 = a RPM²   - b RPM  - c
    DELTA = b**(2) - 4*a*c
    RPM1 = (-b + np.sqrt(DELTA))/(2*a)
    #RPM2 = (-b - np.sqrt(DELTA))/(2*a)
    #print(RPM1, RPM2)
    return RPM1
# 3) Escolha da tensão -> numero de celulas

# 4)
def motor(n_celulas,RPM):
    V = 3.7
    if(n_celulas == 1):
        V = 3.7
    if(n_celulas == 2):
         V = 2*V
    if(n_celulas == 3):
        V = 3*V 
    if(n_celulas == 4):
        V = 4*V
        
    KV = RPM/V
    return KV

# 5) i = pesquisa_na_net(KV_MOTOR)
#6 ESC(V_motor, i_motor)
#7) autonomia = Escolhemos uma aunotomia
#8) 


def Bateria(v,autonomia_em_min,i_usual, imax, numero_de_motores):
    autonomia_em_h = autonomia_em_min/60
    i_usual_total = i_usual*numero_de_motores
    carga = autonomia_em_h*i_usual_total*1000
    C = (imax*numero_de_motores)/(carga/1000)
    # print(" Carga = " +str(carga)+ " mAh " " C = " +str(C))
    resposta = [carga, C]
    return resposta

    


# Calculos iniciais


In [290]:
g = 10
P = 1.7*g
E = empuxo(14208,9.4,5,6,4) 
trust_to_weight_ratio = E/(P) 
print(trust_to_weight_ratio) 

4.025769458447914


In [291]:
RPM(17,4.025769,9.4,5,6,4)

14207.999280651986

In [292]:
RPM(17,4.025769,9.4,5,6,4)

14207.999280651986

In [293]:
motor(4, 14207.999)

959.9999324324324

In [294]:
empuxo_dada_a_tensao(14.8,1290,10,4.7,6,4)

156.5069111957366

In [295]:
empuxo_dada_a_tensao(14.8,1200,10,6,6,4)

157.55093136440254

In [296]:
empuxo_dada_a_tensao(14.8,1200,10,6,6,4)

157.55093136440254

In [408]:
Bateria(14.8,12,15/4,20,4)


[3000.0, 26.666666666666668]

In [409]:
Bateria(14.8,20,20/4,20,4)

[6666.666666666666, 12.000000000000002]

In [415]:
Bateria(11.1,15,20/4,10,4)

[5000.0, 8.0]

# Requisitos de Projeto

## 1) Caluclo do RPM dada um propeller

In [331]:
#  RPM(P,trust_to_weight_ratio , d,pitch, V0, numero_de_motores):
m_bateria_inicial = 554
m_drone = 929 + m_bateria_inicial + 14.5 + 2.7 + 2 + ((105 + 190)/2) + 10 + 250 + 90
m_drone = m_drone/1000
print("Masssa drone = " +str(m_drone)+ " kg ")
print()
g = 10
# valroes uteis:
P_drone = m_drone*g
trust_to_weight_ratio = 4 
V0 = 6
numero_de_motores = 4

d = [9,10,10,8,10,9] 
pitch = [6,5.5,4.5,4.5,4.5,4.7]
rpm = []

# for i in range(len(pitch)):
for i in range(len(pitch)):
    rpm.append(RPM(P,trust_to_weight_ratio , d[i], pitch[i], V0, numero_de_motores))
    print("Velocidade ang =" +str(round(rpm[i])) + " rpm" + " -- opção " + str(i))   
#numero = np.arange(0, len(pitch), 1)
#plt.plot(numero,rpm,color = 'red',linewidth=4,label="Circuito Completo")



Masssa drone = 1.9997 kg 

Velocidade ang =14298.0 rpm -- opção 0
Velocidade ang =12466.0 rpm -- opção 1
Velocidade ang =13355.0 rpm -- opção 2
Velocidade ang =18898.0 rpm -- opção 3
Velocidade ang =13355.0 rpm -- opção 4
Velocidade ang =15476.0 rpm -- opção 5


## 2) Escolha uma tensão
Escolhemos 3s ou 4s , isto é, 11.1 ou 14.8

## 3) Definindo-se o motor pelo KV

In [339]:
# Motor 3S
print("Bateriu 3S")
print()
KV = []
for i in range(len(pitch)):
    KV.append(motor(3,rpm[i]))
    print("KV =" +str(round(KV[i])) + " -- opção " + str(i) ) 
# 4S
print()
print("Bateriu 4S")
print()
KV = []
for i in range(len(pitch)):
    KV.append(motor(4,rpm[i]))
    print("KV =" +str(round(KV[i])) + " -- opção " + str(i) ) 
print()    
print("Comprar com folga pra cima")

Bateriu 3S

KV =1288.0 -- opção 0
KV =1123.0 -- opção 1
KV =1203.0 -- opção 2
KV =1703.0 -- opção 3
KV =1203.0 -- opção 4
KV =1394.0 -- opção 5

Bateriu 4S

KV =966.0 -- opção 0
KV =842.0 -- opção 1
KV =902.0 -- opção 2
KV =1277.0 -- opção 3
KV =902.0 -- opção 4
KV =1046.0 -- opção 5

Comprar com folga pra cima


### 5) i = pesquisa_na_net(KV_MOTOR)
### 6 ESC(V_motor, i_motor)
### 7) autonomia = Escolhemos uma aunotomia
queremos 12 min
### 8) Bateria

In [427]:
autonomia_em_min = 12
# i_usual = "fudeu"
i_usual = 20
v = [14.8,14.8]
imax = [28,20]
numero_de_motores = 4
valores = []
for i in range(len(v)):
    valores.append(Bateria(v[i],autonomia_em_min,i_usual, imax[i], numero_de_motores))
    print("[Carga, C] =" + str(valores[i]) + " mAh , admensional" + " -- opção " + str(i)) 
    
    

[Carga, C] =[16000.0, 7.0] mAh , admensional -- opção 0
[Carga, C] =[16000.0, 5.0] mAh , admensional -- opção 1
